In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
import base64
from io import BytesIO
from PIL import Image
from torchvision import models, transforms
import torch

In [ ]:
import pandas as pd

df = pd.read_csv('НТО.csv')
df

,view,like,comment,hide,expand,open_photo,open,share_to_message,text,photo
0,2694,175,2,2,2,151,73,6,Чудесного всем утра&#33;<br><br>#ShuiLongYin #...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,21688,58,2,8,944,1595,1102,51,&#127802; ЦВЕТЫ ПРОТИВ ВРЕДИТЕЛЕЙ: КРАСОТА С П...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
2,93784,1225,14,34,86,6370,4755,457,"Моя волшебная коробочка, которая помогает заня...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
3,15430,68,11,0,8,1520,428,25,Состояние дороги в деревню Стешевская (Корбозеро),/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
4,13578,284,1,30,4,189,28,9,&#128247;aiartistically,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
...,...,...,...,...,...,...,...,...,...,...
19995,16469,235,8,6,41,650,82,207,"&#9989;Травы, которые понижают давление&#33; В...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
19996,6394,89,0,0,0,72,1,60,NaN,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
19997,6717,196,0,1,0,133,5,27,Лойс если ждёшь &#128077;,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
19998,5501,14,0,0,0,33,1,1,NaN,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   view              20000 non-null  int64 
 1   like              20000 non-null  int64 
 2   comment           20000 non-null  int64 
 3   hide              20000 non-null  int64 
 4   expand            20000 non-null  int64 
 5   open_photo        20000 non-null  int64 
 6   open              20000 non-null  int64 
 7   share_to_message  20000 non-null  int64 
 8   text              16422 non-null  object
 9   photo             20000 non-null  object
dtypes: int64(8), object(2)
memory usage: 1.5+ MB


In [ ]:
# Разделение на два DataFrame
df_with_text = df[df['text'].notna() & (df['text'].str.strip() != '')]
df_without_text = df[df['text'].isna() | (df['text'].str.strip() == '')]

# Проверка размеров
print(f'Количество записей с текстом: {len(df_with_text)}')
print(f'Количество записей без текста: {len(df_without_text)}')

Количество записей с текстом: 16422
Количество записей без текста: 3578


In [ ]:
df_with_text = df_with_text.reset_index()
df_without_text = df_without_text.reset_index()

df_without_text

,index,view,like,comment,hide,expand,open_photo,open,share_to_message,text,photo
0,8,21575,1979,54,4,0,495,40,149,NaN,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,12,9822,396,7,0,0,923,84,110,NaN,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
2,14,48465,1630,11,13,0,1195,33,252,NaN,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
3,17,7911,476,2,7,0,240,40,24,NaN,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
4,21,3071,162,2,4,0,87,40,11,NaN,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
...,...,...,...,...,...,...,...,...,...,...,...
3573,19985,3884,6,0,0,0,85,3,0,NaN,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
3574,19993,3992,114,0,0,0,42,5,6,NaN,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
3575,19996,6394,89,0,0,0,72,1,60,NaN,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
3576,19998,5501,14,0,0,0,33,1,1,NaN,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


# Сначала там где есть текст

In [ ]:
train_X_text = df_with_text[['view', 'text', 'photo']]
train_y_text = df_with_text[['index', 'like', 'comment', 'hide', 'expand', 'open_photo', 'open', 'share_to_message']]

train_X_text.head()

,view,text,photo
0,2694,Чудесного всем утра&#33;<br><br>#ShuiLongYin #...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,21688,&#127802; ЦВЕТЫ ПРОТИВ ВРЕДИТЕЛЕЙ: КРАСОТА С П...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
2,93784,"Моя волшебная коробочка, которая помогает заня...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
3,15430,Состояние дороги в деревню Стешевская (Корбозеро),/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
4,13578,&#128247;aiartistically,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [ ]:
train_X_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16422 entries, 0 to 16421
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   view    16422 non-null  int64 
 1   text    16422 non-null  object
 2   photo   16422 non-null  object
dtypes: int64(1), object(2)
memory usage: 385.0+ KB


In [ ]:
train_y_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16422 entries, 0 to 16421
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   index             16422 non-null  int64
 1   like              16422 non-null  int64
 2   comment           16422 non-null  int64
 3   hide              16422 non-null  int64
 4   expand            16422 non-null  int64
 5   open_photo        16422 non-null  int64
 6   open              16422 non-null  int64
 7   share_to_message  16422 non-null  int64
dtypes: int64(8)
memory usage: 1.0 MB


In [ ]:
train_X_text.loc[4, 'text']

'&#128247;aiartistically'

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words_rus = list(stopwords.words('russian'))
stop_words_rus[0:10]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [ ]:
import re

def clean_text(text, stop_words_rus):
    if pd.isnull(text):
        return ''

    # Удаление ссылок
    text = re.sub(r'https?://\S+|www\.\S+|t\.me/\S+|bit\.ly/\S+', '', text)

    # Удаление HTML-тегов
    text = re.sub(r'<.*?>', '', text)
    # Удаление хештогов с только цифрами
    text = re.sub(r'#[\d\W]+', '', text)

    # Удаление эмодзи
    emoji_pattern = re.compile("[\U0001F600-\U0001F64F"
                               "\U0001F300-\U0001F5FF"
                               "\U0001F680-\U0001F6FF"
                               "\U0001F700-\U0001F77F"
                               "\U0001F780-\U0001F7FF"
                               "\U0001F800-\U0001F8FF"
                               "\U0001F900-\U0001F9FF"
                               "\U0001FA00-\U0001FA6F"
                               "\U0001FA70-\U0001FAFF"
                               "\U00002702-\U000027B0"
                               "\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)

    text = emoji_pattern.sub(r'', text)

    # Удаление пунктуации
    # text = re.sub(r'[^\w\s]', ' ', text)

    # Приведение к нижнему регистру
    text = text.lower()

    # Удаление стоп-слов
    words = text.split()
    text = ' '.join(word for word in words if word not in stop_words_rus)

    return text

# Применение к DataFrame
train_X_text['text_clean'] = train_X_text['text'].apply(lambda x: clean_text(x, stop_words_rus))

train_X_text.head()

<ipython-input-13-f9f1f87c0f16>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_X_text['text_clean'] = train_X_text['text'].apply(lambda x: clean_text(x, stop_words_rus))


,view,text,photo,text_clean
0,2694,Чудесного всем утра&#33;<br><br>#ShuiLongYin #...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,чудесного всем утра&shuilongyin #waterdragonsc...
1,21688,&#127802; ЦВЕТЫ ПРОТИВ ВРЕДИТЕЛЕЙ: КРАСОТА С П...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,&цветы против вредителей: красота пользой &цве...
2,93784,"Моя волшебная коробочка, которая помогает заня...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,"волшебная коробочка, которая помогает занять р..."
3,15430,Состояние дороги в деревню Стешевская (Корбозеро),/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,состояние дороги деревню стешевская (корбозеро)
4,13578,&#128247;aiartistically,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,&aiartistically


In [ ]:
print(train_X_text.loc[5223, 'text'])
print()
train_X_text.loc[5223, 'text_clean']

1 &#1257;зек<br>...&#1210;и, татар баеса, чабатасын т&#1199;рг&#1241; эл&#1241;, имеш. Т&#1257;рттер&#1199;ен кара, &#1241;&#33; Ян&#1241;се, син, тирест&#1241; казынып &#1199;ск&#1241;н х&#1241;ерче Хафизулла малае, артыгын кыланасы&#1187;. &#1240;йе, элде &#1174;&#1241;мил, тик аныкы юк&#1241; кайрысыннан &#1199;релг&#1241;н мескен «аяк савыты» т&#1199;гел, аныкы к&#1199;з чагылган саф алтыннан эшл&#1241;нг&#1241;н сувенир. Б&#1241;л&#1241;к&#1241;й ген&#1241; булса да, байлык билгесе&#33; Тик лыгырдасын, х&#1241;чтер&#1199;ш&#33; Аны&#1187; м&#1241;гън&#1241;сез &#1241;йтемн&#1241;рен&#1241; исебез китк&#1241;н, ди&#33; Кайвакыт кит&#1241; шул. Югыйс&#1241; &#1174;&#1241;мил кебек эшкуарга ыбыр-чыбыр бел&#1241;н булашып вакланмыйча эред&#1241;н ген&#1241; алдыртып яшисе д&#1241;... &#1198;зе&#1187;н&#1241;н т&#1199;б&#1241;нн&#1241;рг&#1241; т&#1257;кереп кен&#1241;... Ш&#1241;п фикерме? Ш&#1241;п&#33; Вакландыра, шайтан. Ч&#1257;нки авылдаш егет бел&#1241;н к&#1257;н д&#1241; бер к

'1 &зек...&и, татар баеса, чабатасын т&рг&эл&имеш. т&рттер&ен кара, &ян&се, син, тирест&казынып &ск&н х&ерче хафизулла малае, артыгын кыланасы&йе, элде &мил, тик аныкы юк&кайрысыннан &релг&н мескен «аяк савыты» т&гел, аныкы к&з чагылган саф алтыннан эшл&нг&н сувенир. б&л&к&й ген&булса да, байлык билгесе&тик лыгырдасын, х&чтер&ш&аны&м&гън&сез &йтемн&рен&исебез китк&н, ди&кайвакыт кит&шул. югыйс&мил кебек эшкуарга ыбыр-чыбыр бел&н булашып вакланмыйча эред&н ген&алдыртып яшисе д&зе&н&н т&б&нн&рг&т&кереп кен&ш&п фикерме? ш&п&вакландыра, шайтан. ч&нки авылдаш егет бел&н к&н д&бер казанда кайный. д&рес, &мил – ху&а, &ул – ярд&мче ген&л&кин &зен &лл&кем итеп тота&килешми, дим&мил, этк&каршы син д&р&урамга ку, нервы&ны ашатып ятма&аман-&аман телб&р гайфиг&бурыч т&л&рг&дим&г&н. й&з гаебен тап, й&зг&ваклан, &н&ил-к&н бел&н идар&итк&н т&р&л&р д&кайчакта тун якасыннан «бет эзли». бурыч н&рс&д&н гыйбар&т иде со&ун&ч яшьлек &мил язгы ташуда боз астына чумгач, аны к&рим атасы гайфи коткарды. аларны&е

In [ ]:
from sentence_transformers import SentenceTransformer

# Загружаем мультиязычную модель
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

# Векторизуем текст
X_text = model.encode(train_X_text['text_clean'], show_progress_bar=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

In [ ]:
import numpy as np

# Сохраняем вектор в файл
# np.save('X_text_vectors.npy', X_text)
X_text = np.load('X_text_vectors.npy')

X_text

array([[-0.02553961,  0.00288923, -0.06962796, ..., -0.05832887,
         0.00131535,  0.018452  ],
       [ 0.06408303,  0.00251951, -0.03679717, ...,  0.00826265,
         0.0186991 ,  0.00334279],
       [-0.0051241 ,  0.05670672, -0.01311661, ...,  0.01497454,
         0.01747109,  0.00788668],
       ...,
       [ 0.0097506 , -0.00943262, -0.0227845 , ...,  0.00274428,
        -0.03615784, -0.01465408],
       [ 0.04854701, -0.05584531, -0.09195642, ..., -0.02147692,
        -0.04589322,  0.0323163 ],
       [ 0.03211773, -0.03897146, -0.03664057, ...,  0.02258866,
        -0.05007548, -0.02632942]], dtype=float32)

In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image
import base64
from io import BytesIO

# Подготовка модели ResNet
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet = models.resnet18(pretrained=True)
resnet.fc = torch.nn.Identity()
resnet = resnet.to(device)
resnet.eval()

# Трансформации для изображения
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Декодирование base64 и извлечение фич
def extract_image_features(base64_str):
    try:
        img_data = base64.b64decode(base64_str)
        img = Image.open(BytesIO(img_data)).convert('RGB')
        img = transform(img).unsqueeze(0).to(device)
        with torch.no_grad():
            features = resnet(img).squeeze().cpu().numpy()
        return features
    except Exception as e:
        return np.zeros(512)

# Извлечение фич из изображений
X_image = np.stack(df_with_text['photo'].apply(extract_image_features))

# Сохраняем результат
np.save('X_image_vectors.npy', X_image)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
X_image.shape

(16422, 512)

In [ ]:
X_text.shape

(16422, 512)

In [ ]:
df_with_text[['view']].values.shape

(16422, 1)

In [ ]:
# Фичи для постов с текстом
X_full_text = np.hstack([X_text,
                    X_image,
                    df_with_text[['view']].values])

y_full_text = df_with_text[['like', 'comment', 'hide', 'expand', 'open_photo', 'open', 'share_to_message']].values

print(X_full_text.shape, y_full_text.shape)

(16422, 1025) (16422, 7)


In [ ]:
!pip install catboost --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 11.5 MB/s eta 0:00:00


In [ ]:
%%time
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score, mean_squared_error

# Разделение данных на обучение и валидацию
X_train_text, X_val_text, y_train_text, y_val_text = train_test_split(X_full_text, y_full_text, test_size=0.2, random_state=42)

model = MultiOutputRegressor(CatBoostRegressor(
    iterations=200,
    learning_rate=0.05,
    verbose=0,
    task_type='GPU'  # Добавляем использование GPU
))

model.fit(X_train_text, y_train_text)

# Оценка на валидации
y_pred_text = model.predict(X_val_text)
mse = mean_squared_error(y_val_text, y_pred_text, multioutput='raw_values')
print(f'MSE по метрикам: {mse}')

r2 = r2_score(y_val_text, y_pred_text, multioutput='raw_values')
print(f'R² по метрикам: {r2}')

print(np.mean(r2))

MSE по метрикам: [5.85942728e+05 4.77284468e+03 8.72424517e+02 6.93391822e+07
 3.18841021e+07 8.83580371e+07 2.09449595e+04]


NameError: name 'r2_score' is not defined

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

r2 = r2_score(y_val_text, y_pred_text, multioutput='raw_values')
print(f'R² по метрикам: {r2}')

np.mean(r2)

R² по метрикам: [0.26443833 0.03128871 0.38700332 0.27820767 0.18417359 0.16155086
 0.30642725]


0.23044139110523468

# Теперь только где фото

In [ ]:
train_X_image = df_without_text[['view', 'photo']]
train_y_image = df_without_text[['like', 'comment', 'hide', 'expand', 'open_photo', 'open', 'share_to_message']]

train_X_image.head()

,view,photo
0,21575,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,9822,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
2,48465,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
3,7911,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
4,3071,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [ ]:
train_X_image.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3578 entries, 0 to 3577
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   view    3578 non-null   int64 
 1   photo   3578 non-null   object
dtypes: int64(1), object(1)
memory usage: 56.0+ KB


In [ ]:
train_y_image.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3578 entries, 0 to 3577
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   like              3578 non-null   int64
 1   comment           3578 non-null   int64
 2   hide              3578 non-null   int64
 3   expand            3578 non-null   int64
 4   open_photo        3578 non-null   int64
 5   open              3578 non-null   int64
 6   share_to_message  3578 non-null   int64
dtypes: int64(7)
memory usage: 195.8 KB


In [ ]:
# Извлечение фич из изображений
X_image_only = np.stack(train_X_image['photo'].apply(extract_image_features))

# Сохраняем результат
np.save('X_image_only_vectors.npy', X_image_only)

In [ ]:
X_image_only.shape

(3578, 512)

In [ ]:
# Фичи для постов с текстом
X_full_image = np.hstack([X_image_only,
                          train_X_image[['view']].values])

y_full_image = train_y_image.values

print(X_full_image.shape, y_full_image.shape)

(3578, 513) (3578, 7)


In [ ]:
print(X_full_text.shape, y_full_text.shape)

(16422, 1025) (16422, 7)


In [ ]:
%%time
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score, mean_squared_error

# Разделение данных на обучение и валидацию
X_train_image, X_val_image, y_train_image, y_val_image = train_test_split(X_full_image,
                                                                          y_full_image,
                                                                          test_size=0.2,
                                                                          random_state=42)

model_on_imgs = MultiOutputRegressor(CatBoostRegressor(
    iterations=200,
    learning_rate=0.05,
    verbose=0,
    task_type='GPU'  # Добавляем использование GPU
))

model_on_imgs.fit(X_train_image, y_train_image)

# Оценка на валидации
y_pred_image = model_on_imgs.predict(X_val_image)
mse = mean_squared_error(y_val_image, y_pred_image, multioutput='raw_values')
print(f'MSE по метрикам: {mse}')

r2 = r2_score(y_val_image, y_pred_image, multioutput='raw_values')
print(f'R² по метрикам: {r2}')

print(np.mean(r2))

MSE по метрикам: [1.35680629e+06 3.04296891e+02 1.67214102e+02 9.20871789e+02
 4.00548078e+06 3.47085260e+05 3.77593986e+04]
R² по метрикам: [ 0.44917637  0.31633736  0.51134767 -0.00139891  0.20693317  0.04870437
  0.34575444]
0.2681220675558384
CPU times: user 28.4 s, sys: 8.44 s, total: 36.8 s
Wall time: 22.3 s


In [ ]:
%%time
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score, mean_squared_error

# Разделение данных на обучение и валидацию
X_train_image, X_val_image, y_train_image, y_val_image = train_test_split(X_full_image,
                                                                          y_full_image,
                                                                          test_size=0.2,
                                                                          random_state=42)

from lightgbm import LGBMRegressor

model_on_imgs_LGBM = MultiOutputRegressor(LGBMRegressor(
    n_estimators=200,
    learning_rate=0.05,
    device='gpu'
))


model_on_imgs_LGBM.fit(X_train_image, y_train_image)

# Оценка на валидации
y_pred_image = model_on_imgs_LGBM.predict(X_val_image)
mse = mean_squared_error(y_val_image, y_pred_image, multioutput='raw_values')
print(f'MSE по метрикам: {mse}')

r2 = r2_score(y_val_image, y_pred_image, multioutput='raw_values')
print(f'R² по метрикам: {r2}')

print(np.mean(r2))

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 130815
[LightGBM] [Info] Number of data points in the train set: 2862, number of used features: 513


LightGBMError: No OpenCL device found

# Вова

In [ ]:
from PIL import Image
from io import BytesIO
import base64

import pandas as pd
import numpy as np
import cv2

from sklearn.feature_extraction.text import TfidfVectorizer
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor


from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split

from transformers import DistilBertTokenizer, DistilBertModel

import re
import nltk
from nltk.corpus import stopwords

import xgboost as xgb
import torch
import torchvision.models as models
import torchvision.transforms as transforms

import pickle

from sentence_transformers import SentenceTransformer

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

import matplotlib.pyplot as mlt

In [ ]:
train = pd.read_csv("НТО.csv")

In [ ]:
train['text'] = train['text'].fillna('')

In [ ]:
train_NaN = train[train['text'] == ''].reset_index()
train_no_NaN = train[train['text'] != ''].reset_index()

In [ ]:
names_columns = ["like", "comment", "hide", "expand", "open_photo", "open", "share_to_message"]

In [ ]:
for cln in names_columns:
    tdp_no_NaN = train_no_NaN.agg({
       cln: 'mean'
    })
    tdp_NaN = train_NaN.agg({
       cln: 'mean'
    })
    print(tdp_no_NaN)
    print(tdp_NaN)

like    312.739252
dtype: float64
like    692.735327
dtype: float64
comment    10.805444
dtype: float64
comment    6.970654
dtype: float64
hide    10.712033
dtype: float64
hide    9.455003
dtype: float64
expand    908.004932
dtype: float64
expand    0.227781
dtype: float64
open_photo    880.154488
dtype: float64
open_photo    787.859978
dtype: float64
open    617.868956
dtype: float64
open    175.572946
dtype: float64
share_to_message    48.236451
dtype: float64
share_to_message    95.408888
dtype: float64


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words_rus = list(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def clean_text(text, stop_words_rus):
    if pd.isnull(text):
        return ''

    text = re.sub(r'https?://\S+|www\.\S+|t\.me/\S+|bit\.ly/\S+', '', text)

    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'#[\d\W]+', '', text)

    emoji_pattern = re.compile("[\U0001F600-\U0001F64F"
                               "\U0001F300-\U0001F5FF"
                               "\U0001F680-\U0001F6FF"
                               "\U0001F700-\U0001F77F"
                               "\U0001F780-\U0001F7FF"
                               "\U0001F800-\U0001F8FF"
                               "\U0001F900-\U0001F9FF"
                               "\U0001FA00-\U0001FA6F"
                               "\U0001FA70-\U0001FAFF"
                               "\U00002702-\U000027B0"
                               "\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)

    text = emoji_pattern.sub(r'', text)

    text = text.lower()

    words = text.split()
    text = ' '.join(word for word in words if word not in stop_words_rus)

    return text


train_no_NaN['text_clean'] = train_no_NaN['text'].apply(lambda x: clean_text(x, stop_words_rus))

train_no_NaN.head()

,index,view,like,comment,hide,expand,open_photo,open,share_to_message,text,photo,text_clean
0,0,2694,175,2,2,2,151,73,6,Чудесного всем утра&#33;<br><br>#ShuiLongYin #...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,чудесного всем утра&shuilongyin #waterdragonsc...
1,1,21688,58,2,8,944,1595,1102,51,&#127802; ЦВЕТЫ ПРОТИВ ВРЕДИТЕЛЕЙ: КРАСОТА С П...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,&цветы против вредителей: красота пользой &цве...
2,2,93784,1225,14,34,86,6370,4755,457,"Моя волшебная коробочка, которая помогает заня...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,"волшебная коробочка, которая помогает занять р..."
3,3,15430,68,11,0,8,1520,428,25,Состояние дороги в деревню Стешевская (Корбозеро),/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,состояние дороги деревню стешевская (корбозеро)
4,4,13578,284,1,30,4,189,28,9,&#128247;aiartistically,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,&aiartistically


In [ ]:
vectorizer = TfidfVectorizer(max_features=1000)
X_text = vectorizer.fit_transform(train_no_NaN['text_clean']).toarray()

with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

In [ ]:
def get_image_features(photo):
    if pd.isna(photo):
        return np.zeros(32)

    try:
        image_data = base64.b64decode(photo)
        image = Image.open(BytesIO(image_data)).convert('RGB')
        image_np = np.array(image)

        image_cv = cv2.cvtColor(image_np, cv2.COLOR_RGB2HSV)
        hue_hist = cv2.calcHist([image_cv], [0], None, [16], [0, 180])
        sat_hist = cv2.calcHist([image_cv], [1], None, [8], [0, 256])
        val_hist = cv2.calcHist([image_cv], [2], None, [8], [0, 256])

        hue_hist = hue_hist / (hue_hist.sum() + 1e-6)
        sat_hist = sat_hist / (sat_hist.sum() + 1e-6)
        val_hist = val_hist / (val_hist.sum() + 1e-6)

        features = np.concatenate([hue_hist, sat_hist, val_hist]).flatten()
        return features
    except Exception as e:
        print(f"Ошибка обработки изображения: {e}")
        return np.zeros(32)

X_image = np.array([get_image_features(photo) for photo in train_no_NaN['photo']])
X_image_NaN = np.array([get_image_features(photo) for photo in train_NaN['photo']])

In [ ]:
train_X_no_NaN = pd.DataFrame(np.hstack((X_text, X_image)))

final_train_X_no_NaN = pd.concat([train_no_NaN, train_X_no_NaN], axis=1)
final_train_X_no_NaN = final_train_X_no_NaN.drop(columns=['text', 'photo', 'text_clean'])

final_train_X_no_NaN

,index,view,like,comment,hide,expand,open_photo,open,share_to_message,0,...,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031
0,0,2694,175,2,2,2,151,73,6,0.0,...,0.000881,0.000548,0.017316,0.197837,0.222656,0.182689,0.174709,0.105993,0.055521,0.043278
1,1,21688,58,2,8,944,1595,1102,51,0.0,...,0.209770,0.559792,0.035359,0.086669,0.082218,0.101002,0.130556,0.152344,0.165930,0.245923
2,2,93784,1225,14,34,86,6370,4755,457,0.0,...,0.037313,0.068197,0.019938,0.036275,0.074707,0.095296,0.111959,0.207296,0.224772,0.229757
3,3,15430,68,11,0,8,1520,428,25,0.0,...,0.000000,0.000000,0.000828,0.026483,0.193790,0.237421,0.247583,0.153569,0.086864,0.053463
4,4,13578,284,1,30,4,189,28,9,0.0,...,0.064192,0.060009,0.204724,0.296260,0.173582,0.105253,0.085461,0.059747,0.043322,0.031650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16417,19991,11894,313,24,2,112,2254,1850,84,0.0,...,0.021022,0.003444,0.000000,0.021399,0.094555,0.242307,0.252656,0.202971,0.061712,0.124400
16418,19992,56263,249,49,16,3806,0,237,130,0.0,...,0.104248,0.129013,0.031815,0.328400,0.135010,0.178619,0.110199,0.027313,0.024597,0.164047
16419,19994,2200,3,0,0,86,403,292,3,0.0,...,0.012730,0.013143,0.107514,0.105836,0.161328,0.224196,0.146255,0.084076,0.165464,0.005331
16420,19995,16469,235,8,6,41,650,82,207,0.0,...,0.013976,0.013242,0.029942,0.030300,0.028007,0.023617,0.031035,0.057411,0.082873,0.716815


In [ ]:
final_train_X_NaN = pd.concat([train_NaN, pd.DataFrame(X_image_NaN)], axis=1)
final_train_X_NaN = final_train_X_NaN.drop(columns=['text', 'photo'])

final_train_X_NaN

,index,view,like,comment,hide,expand,open_photo,open,share_to_message,0,...,22,23,24,25,26,27,28,29,30,31
0,8,21575,1979,54,4,0,495,40,149,0.145875,...,0.016659,0.065341,0.026525,0.014220,0.010514,0.009480,0.012830,0.026016,0.046921,0.853492
1,12,9822,396,7,0,0,923,84,110,0.240291,...,0.035269,0.076646,0.058047,0.044850,0.057404,0.087947,0.097528,0.128488,0.185957,0.339779
2,14,48465,1630,11,13,0,1195,33,252,1.000000,...,0.000000,0.000000,0.005310,0.190186,0.282817,0.068583,0.050517,0.057983,0.077881,0.266724
3,17,7911,476,2,7,0,240,40,24,0.244610,...,0.055211,0.048839,0.085545,0.231429,0.193741,0.177200,0.149856,0.094514,0.036771,0.030944
4,21,3071,162,2,4,0,87,40,11,0.000241,...,0.058861,0.011157,0.000508,0.029618,0.118472,0.197239,0.188972,0.156946,0.130485,0.177761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3573,19985,3884,6,0,0,0,85,3,0,0.246603,...,0.010888,0.042289,0.199389,0.084484,0.067501,0.092674,0.173158,0.193407,0.146607,0.042780
3574,19993,3992,114,0,0,0,42,5,6,1.000000,...,0.000000,0.000000,0.001221,0.014811,0.027311,0.023910,0.022542,0.020654,0.025374,0.864176
3575,19996,6394,89,0,0,0,72,1,60,0.722326,...,0.002583,0.019159,0.127335,0.255911,0.137442,0.039435,0.030713,0.039579,0.052207,0.317378
3576,19998,5501,14,0,0,0,33,1,1,0.120978,...,0.009782,0.016800,0.035156,0.036762,0.060299,0.091764,0.400159,0.242800,0.063592,0.069468


In [ ]:
train_y = final_train_X_no_NaN[names_columns]
train_X = final_train_X_no_NaN.drop(columns=names_columns)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.2, random_state=42)

In [ ]:
model_no_NaN = CatBoostRegressor(
    iterations=5000,
    learning_rate=0.05,
    depth=8,
    loss_function='MultiRMSE',
    ignored_features=['index'],
    task_type='GPU',           # Включаем GPU
    devices='0',               # Если у тебя несколько GPU, можешь указать, какую использовать
    boosting_type='Plain',      # Важный параметр для работы на GPU!
    verbose=0
)

model_no_NaN.fit(X_train, y_train)

0:	learn: 11392.8455159	total: 168ms	remaining: 13m 59s
1:	learn: 11235.3833867	total: 346ms	remaining: 14m 24s
2:	learn: 11097.5361711	total: 476ms	remaining: 13m 13s
3:	learn: 11016.8967272	total: 574ms	remaining: 11m 56s
4:	learn: 10887.6072792	total: 667ms	remaining: 11m 6s
5:	learn: 10755.7065205	total: 759ms	remaining: 10m 31s
6:	learn: 10617.4738015	total: 866ms	remaining: 10m 17s
7:	learn: 10477.4051026	total: 970ms	remaining: 10m 4s
8:	learn: 10358.4517516	total: 1.05s	remaining: 9m 45s
9:	learn: 10280.7140066	total: 1.16s	remaining: 9m 36s
10:	learn: 10163.5876871	total: 1.29s	remaining: 9m 46s
11:	learn: 10087.4367808	total: 1.37s	remaining: 9m 31s
12:	learn: 10014.3074150	total: 1.48s	remaining: 9m 27s
13:	learn: 9911.5869827	total: 1.56s	remaining: 9m 16s
14:	learn: 9804.6295692	total: 1.67s	remaining: 9m 15s
15:	learn: 9752.0714669	total: 1.83s	remaining: 9m 29s
16:	learn: 9706.5771816	total: 1.98s	remaining: 9m 41s
17:	learn: 9636.0532370	total: 2.09s	remaining: 9m 38s
1

In [ ]:
y_pred_no_NaN = model_no_NaN.predict(X_val)

y_pred_no_NaN = y_pred_no_NaN * (y_pred_no_NaN > 0)

mse = mean_squared_error(y_val, y_pred_no_NaN, multioutput='raw_values')
print(f'MSE по метрикам: {mse}')

r2 = r2_score(y_val, y_pred_no_NaN, multioutput='raw_values')
print(f'R² по метрикам: {r2}')

MSE по метрикам: [6.59388823e+05 5.36344429e+03 1.01609337e+03 8.01131612e+07
 2.96135092e+07 9.09300482e+07 2.69847370e+04]
R² по метрикам: [ 0.17223797 -0.08858121  0.28605645  0.16605499  0.24227181  0.13714447
  0.10642567]


In [ ]:
y_pred_no_NaN = model_no_NaN.predict(train_X)

y_pred_no_NaN = pd.DataFrame(y_pred_no_NaN)
y_pred_no_NaN.columns = names_columns

In [ ]:
model_no_NaN = CatBoostRegressor(
    iterations=5000,
    learning_rate=0.05,
    depth=8,
    loss_function='MultiRMSE',
    ignored_features=['index'],
    task_type='GPU',           # Включаем GPU
    devices='0',               # Если у тебя несколько GPU, можешь указать, какую использовать
    boosting_type='Plain',      # Важный параметр для работы на GPU!
    verbose=0
)

model_no_NaN.fit(train_X, train_y)

model_no_NaN.save_model('catboost_model_no_NaN.cbm')

In [ ]:
train_y = final_train_X_NaN[names_columns]
train_X = final_train_X_NaN.drop(columns=names_columns)

X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.2, random_state=42)

model_NaN = CatBoostRegressor(
    iterations=5000,
    learning_rate=0.05,
    depth=8,
    loss_function='MultiRMSE',
    ignored_features=['index'],
    task_type='GPU',           # Включаем GPU
    devices='0',               # Если у тебя несколько GPU, можешь указать, какую использовать
    boosting_type='Plain',      # Важный параметр для работы на GPU!
    verbose=1000
)

model_NaN.fit(X_train, y_train)

0:	learn: 4778.8018849	total: 56.5ms	remaining: 4m 42s
1000:	learn: 260.6348858	total: 16s	remaining: 1m 3s
2000:	learn: 78.2928438	total: 40.3s	remaining: 1m
3000:	learn: 27.1257328	total: 1m 6s	remaining: 44.2s
4000:	learn: 10.3991955	total: 1m 24s	remaining: 21.2s
4999:	learn: 4.2884518	total: 1m 43s	remaining: 0us


In [ ]:
y_pred_NaN = model_NaN.predict(X_val)

y_pred_NaN = y_pred_NaN * (y_pred_NaN > 0)

mse = mean_squared_error(y_val, y_pred_NaN, multioutput='raw_values')
print(f'MSE по метрикам: {mse}')

r2 = r2_score(y_val, y_pred_NaN, multioutput='raw_values')
print(f'R² по метрикам: {r2}')

MSE по метрикам: [1.42398378e+06 3.18718449e+02 2.25125156e+02 9.20867697e+02
 4.52133834e+06 3.08538757e+05 4.90082407e+04]
R² по метрикам: [ 0.42190427  0.2839365   0.34211331 -0.00139446  0.10479573  0.15435311
  0.15084919]


In [ ]:
y_pred_NaN = model_NaN.predict(train_X)

y_pred_NaN = pd.DataFrame(y_pred_NaN)
y_pred_NaN.columns = names_columns

y_pred_NaN.shape

(3578, 7)

In [ ]:
y_pred_no_NaN.shape

(16422, 7)

In [ ]:
train_NaN.shape

(3578, 11)

In [ ]:
y_pred_NaN.shape

(3578, 7)

In [ ]:
y_pred_NaN['index'] = train_NaN['index']

y_pred_no_NaN['index'] = train_no_NaN['index']

y_pred = pd.concat([y_pred_NaN, y_pred_no_NaN], axis=0).sort_values(by='index').reset_index(drop=True)

In [ ]:
model_NaN = CatBoostRegressor(
    iterations=5000,
    learning_rate=0.05,
    depth=8,
    loss_function='MultiRMSE',
    ignored_features=['index'],
    task_type='GPU',           # Включаем GPU
    devices='0',               # Если у тебя несколько GPU, можешь указать, какую использовать
    boosting_type='Plain',      # Важный параметр для работы на GPU!
    verbose=1000
)

model_NaN.fit(train_X, train_y)

model_NaN.save_model('catboost_model_NaN.cbm')

0:	learn: 4456.3398537	total: 34.5ms	remaining: 2m 52s
1000:	learn: 312.3059943	total: 30.2s	remaining: 2m
2000:	learn: 110.5334289	total: 50.6s	remaining: 1m 15s
3000:	learn: 44.5320147	total: 1m 6s	remaining: 44.6s
4000:	learn: 19.4832012	total: 1m 24s	remaining: 21s
4999:	learn: 8.7754790	total: 1m 41s	remaining: 0us


In [ ]:
y_pred.shape

(20000, 8)

In [ ]:
y_val = train[names_columns]

mse = mean_squared_error(y_val, y_pred[names_columns], multioutput='raw_values')
print(f'MSE по метрикам: {mse}')

r2 = r2_score(y_val, y_pred[names_columns], multioutput='raw_values')
print(f'R² по метрикам: {r2}')

MSE по метрикам: [2.20776171e+05 1.80558239e+03 3.08008942e+02 1.32787224e+07
 5.22626950e+06 1.50139641e+07 8.59534714e+03]
R² по метрикам: [0.88457209 0.62668381 0.83005783 0.83106048 0.7943667  0.37709315
 0.905679  ]


In [ ]:
np.mean(r2)

0.7499304374546617